In [1]:
import taxcalc as tc
import pandas as pd
import numpy as np
import helpers
import copy
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.io import show, output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
start_yr = 2018
end_yr = 2027
years = [year for year in range(start_yr, end_yr + 1)]

# Baseline Calculator

In [3]:
rec_base = tc.Records.cps_constructor()
pol_base = tc.Policy()
calc_base = tc.Calculator(records=rec_base, policy=pol_base)
calc_base.advance_to_year(start_yr)
calc_base.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


# Tax and Benefit Reform

In [4]:
rec_reform = tc.Records.cps_constructor()
pol_reform = tc.Policy()
# read in and implement reform parameters
params = tc.Calculator.read_json_param_objects('tax_reform.json',
                                               assump=None)
pol_reform.implement_reform(params['policy'])
calc_reform = tc.Calculator(records=rec_reform, policy=pol_reform)
calc_reform.advance_to_year(start_yr)
calc_reform.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


# Revenue Raised

In [5]:
tax_rev = calc_reform.weighted_total('combined') - calc_base.weighted_total('combined')
ben_rev = calc_base.weighted_total('benefit_cost_total') - calc_reform.weighted_total('benefit_cost_total')
total_rev = tax_rev + ben_rev

In [6]:
print('Revenue Raised for 2018')
print(f'Total Tax Revenue Raised: ${round(tax_rev, 2)}')
print(f'Total Benefit Revenue Raised: ${round(ben_rev, 2)}')
print('-' * 46)
print(f'Total Revenue Raised: ${round(total_rev, 2)}')

Revenue Raised for 2018
Total Tax Revenue Raised: $488649081187.16
Total Benefit Revenue Raised: $2740908107827.55
----------------------------------------------
Total Revenue Raised: $3229557189014.7


# Initial UBI

Caculations for a UBI in 2018

In [7]:
u18 = calc_reform.weighted_total('nu18')
abv18 = calc_reform.weighted_total('n1820') + calc_reform.weighted_total('n21')
ubi_18, ubi_u18 = helpers.ubi_amount(total_rev, u18, abv18)
print(f'Base UBI for those under 18: ${round(ubi_u18, 2)}')
print(f'Base UBI for those above 18: ${round(ubi_18, 1)}')
print('Note: these do not account for additional tax revenue due to the UBI')

Base UBI for those under 18: $5499.9
Base UBI for those above 18: $10999.8
Note: these do not account for additional tax revenue due to the UBI


In [9]:
# calculator to find initial UBI tax revenue
rec_init = tc.Records.cps_constructor()
pol_init = tc.Policy()
pol_init.implement_reform(params['policy'])
ubi_reform = {
    2018: {
        '_UBI_u18': [ubi_u18],
        '_UBI_1820': [ubi_18],
        '_UBI_21': [ubi_18]
    }
}
pol_init.implement_reform(ubi_reform)
calc_init = tc.Calculator(records=rec_init, policy=pol_init)
calc_init.advance_to_year(2018)
calc_init.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [10]:
reform_rev = calc_reform.weighted_total('combined')
ubi_tax_rev_init = (calc_init.weighted_total('combined') -
                   reform_rev)
print(f'Initial tax revenue raised by a UBI: ${round(ubi_tax_rev_init, 2)}')

Initial tax revenue raised by a UBI: $528323923334.34


In [11]:
# find UBI after accounting for additional tax revenue
ubi_18_final, ubi_u18_final = helpers.ubi_finder(u18, abv18, ubi_18, ubi_u18,
                                                 params, total_rev, reform_rev,
                                                 2018, 100)

2018
Iterations: 15
Revenue Difference: $-40.07


In [12]:
print('Budget Neutral UBI Amounts:')
print(f'\t Those under 18: ${round(ubi_u18_final, 2)}')
print(f'\t Those above 18: ${round(ubi_18_final, 2)}')

Budget Neutral UBI Amounts:
	 Those under 18: $6606.85
	 Those above 18: $13213.7


# Calculations with a UBI

In [13]:
rec_ubi = tc.Records.cps_constructor()
pol_ubi = tc.Policy()
pol_ubi.implement_reform(params['policy'])
ubi_reform = {
    2018: {
        '_UBI_u18': [ubi_u18_final],
        '_UBI_1820': [ubi_18_final],
        '_UBI_21': [ubi_18_final]
    }
}
pol_ubi.implement_reform(ubi_reform)
calc_ubi = tc.Calculator(records=rec_ubi, policy = pol_ubi)
calc_ubi.advance_to_year(2018)
calc_ubi.calc_all()

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [14]:
# UBI table data prep
table_data = calc_ubi.dataframe(['ubi', 'e00200', 'nu18', 'n1820', 'n21',
                                 'combined', 's006', 'benefit_cost_total',
                                 'benefit_value_total', 'elderly_dependent',
                                 'age_head', 'age_spouse'])
mtrs = calc_ubi.mtr()
table_data['pmtr'] = mtrs[0]
table_data['imtr'] = mtrs[1]
table_data['cmtr'] = mtrs[2]
table_data['ppl'] = table_data[['nu18', 'n1820', 'n21']].sum(axis=0)
table_data['ubi/person'] = table_data['ubi'] / table_data['ppl']
tax_change = calc_ubi.array('combined') - calc_base.array('combined')
table_data['tax_change'] = tax_change
ben_change = (calc_ubi.array('benefit_cost_total') -
              calc_base.array('benefit_cost_total'))
table_data['ben_change'] = ben_change
ben_change_welfare = (calc_ubi.array('benefit_value_total') -
                      calc_base.array('benefit_value_total'))
table_data['ben_change_welfare'] = ben_change_welfare
combined_change = (table_data['ubi'] -
                   table_data['tax_change'] +
                   table_data['ben_change'])
table_data['combined_change'] = combined_change
combined_change_welfare = (table_data['ubi'] -
                           table_data['tax_change'] +
                           table_data['ben_change_welfare'])
table_data['combined_change_welfare'] = combined_change_welfare
table_data['65_plus'] = np.where((table_data['elderly_dependent'] == 1) |
                                  (table_data['age_head'] >= 65) |
                                  (table_data['age_spouse'] >= 65), 1, 0)

# All Tax Units

In [15]:
helpers.table(table_data, 'e00200', 'webapp')  # will need to change 'webapp' to 'standard'

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,45.954719,0.0,0,14.2%,13.3%,27.5%,"2,974","19,676","-39,286","-39,286","-22,584","-22,584"
1,0,15.914526,0.0,0,14.2%,10.3%,24.5%,"2,974","16,907","-11,644","-11,644","2,289","2,289"
2,10,14.414201,0.0,0,14.2%,12.1%,26.3%,"5,543","20,383","-11,620","-11,620","3,220","3,220"
3,20,15.253706,0.0,0,14.2%,14.8%,29.0%,"6,201","21,381","-9,220","-9,220","5,960","5,960"
4,30,12.411297,0.0,0,14.2%,18.2%,32.4%,"6,476","22,027","-7,584","-7,584","7,967","7,967"
5,40,10.622800,0.0,0,14.2%,18.8%,33.0%,"6,999","23,218","-7,133","-7,133","9,086","9,086"
6,50,18.995208,0.0,0,14.2%,20.4%,34.6%,"8,111","24,941","-6,129","-6,129","10,701","10,701"
7,75,12.312157,0.0,0,14.2%,21.1%,35.3%,"10,448","27,773","-4,785","-4,785","12,541","12,541"
8,100,18.660672,0.0,0,12.6%,22.3%,34.9%,"13,239","30,858","-4,099","-4,099","13,520","13,520"
9,200,4.954667,0.0,0,7.9%,26.9%,34.8%,"17,099","32,668","-3,755","-3,755","11,813","11,813"


# Tax Units with Someone 65 or Older

In [16]:
elderly_data = copy.deepcopy(table_data[table_data['65_plus'] == 1])

In [17]:
helpers.table(elderly_data, 'e00200', 'webapp')  # will need to change 'webapp' to 'standard'

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,25.842219,0.0,0,14.2%,14.4%,28.6%,"2,559","19,347","-47,220","-47,220","-30,433","-30,433"
1,0,1.599303,0.0,0,14.2%,13.1%,27.3%,"3,628","21,524","-43,137","-43,137","-25,241","-25,241"
2,10,1.413394,0.0,0,14.2%,13.1%,27.3%,"4,623","22,429","-40,500","-40,500","-22,694","-22,694"
3,20,1.208057,0.0,0,14.2%,14.6%,28.8%,"5,097","23,322","-37,524","-37,524","-19,299","-19,299"
4,30,0.931875,0.0,0,14.2%,16.7%,30.9%,"5,265","24,119","-33,561","-33,561","-14,707","-14,707"
5,40,0.817449,0.0,0,14.2%,17.8%,32.0%,"5,316","24,656","-36,161","-36,161","-16,820","-16,820"
6,50,1.552293,0.0,0,14.2%,20.5%,34.7%,"6,070","25,124","-33,428","-33,428","-14,374","-14,374"
7,75,0.874385,0.0,0,14.2%,21.1%,35.3%,"7,804","26,398","-28,962","-28,962","-10,368","-10,368"
8,100,1.333232,0.0,0,12.3%,22.7%,35.0%,"9,097","27,483","-28,514","-28,514","-10,128","-10,128"
9,200,0.384326,0.0,0,8.0%,27.6%,35.5%,"11,877","28,342","-24,673","-24,673","-8,208","-8,208"


# Tax Units without Someone 65 or Older

In [18]:
non_elderly_data = copy.deepcopy(table_data[table_data['65_plus'] == 0])

In [19]:
helpers.table(non_elderly_data, 'e00200', 'webapp')  # will need to change 'webapp' to 'standard'

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg UBI Per Person,Avg MTR - Payroll,Avg MTR - Individual Income,Avg MTR - Combined,Avg Tax Change,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Benefits Change - Welfare Adj,Avg Combined Change,Avg Combined Change - Welfare Adj
0,0,20.112499,0.0,0,14.1%,11.9%,26.1%,"3,508","20,099","-29,091","-29,091","-12,500","-12,500"
1,0,14.315223,0.0,0,14.2%,10.0%,24.2%,"2,901","16,391","-8,125","-8,125","5,365","5,365"
2,10,13.000807,0.0,0,14.2%,12.0%,26.2%,"5,643","20,161","-8,480","-8,480","6,037","6,037"
3,20,14.045649,0.0,0,14.2%,14.8%,29.0%,"6,296","21,214","-6,786","-6,786","8,132","8,132"
4,30,11.479422,0.0,0,14.2%,18.3%,32.5%,"6,575","21,858","-5,476","-5,476","9,807","9,807"
5,40,9.805351,0.0,0,14.2%,18.9%,33.1%,"7,139","23,098","-4,713","-4,713","11,246","11,246"
6,50,17.442915,0.0,0,14.2%,20.4%,34.6%,"8,292","24,925","-3,700","-3,700","12,932","12,932"
7,75,11.437771,0.0,0,14.2%,21.2%,35.3%,"10,650","27,878","-2,936","-2,936","14,292","14,292"
8,100,17.327440,0.0,0,12.6%,22.3%,34.9%,"13,558","31,118","-2,221","-2,221","15,339","15,339"
9,200,4.570341,0.0,0,7.9%,26.8%,34.7%,"17,539","33,031","-1,996","-1,996","13,497","13,497"
